## setup

In [38]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import rasterio as rio
import pandas as pd
import rasterstats
from pathlib import Path
from datetime import datetime

In [4]:
cd /home/micromamba/data

/home/micromamba/data


In [18]:
def read_raster(path,bands=1,crs=False):
    # From rasterio docs with modifications
    with rio.open(path) as dst:
        array = dst.read(bands)
        profile = dst.profile
        crs_val = dst.crs
        # array = np.moveaxis(array,0,-1)
    
    result = [array, profile]

    if crs:
        result.append(crs_val)

    return result


def write_raster(array,profile,out_path,nodata,dtype):
    # From rasterio docs:
    # Register GDAL format drivers and configuration options with a
    # context manager.
    with rio.Env():
        # And then change the band count to 1, set the
        # dtype to uint8, and specify LZW compression.
        profile.update(
            dtype=dtype,
            count=1,
            nodata=nodata,
            compress='lzw')

        with rio.open(out_path, 'w', **profile) as dst:
            dst.write(array.astype(dtype), 1)

    return out_path

def calculate_depth(df,alpha,gamma):

    area = df.geometry.area
    volume = alpha * (area**gamma)
    avg_depth = volume / area
    
    return avg_depth

## planning

steps:

* load data, associate xsection with landslide, velocities
* buffer xsection, get zonal sum and count
* use these properties to get average movement vector
* project this vector into the direction perpendicular to the xsection
    * calculate a perpendicular (downhill) unit vector of the xsection
        * **make sure downhill is 90 deg clockwise of xsection heading**
    * project average velocity vector along this unit vector
    * calculate magnitude of this vector
* get xsection length
* add length and projected velocity attribute to landslide outline
* use power law relationship to calculate landslide volume
* divide volume by area for average depth
* multiply depth * length * velocity to get annual sediment contribution

## load_data

In [6]:
ls = gpd.read_file('landslides/landslide_sediment.shp')
xsections = gpd.read_file('landslides/landslide_sediment_xsections.shp')

ls['velocity'] = np.nan
ls['length'] = np.nan

## calculate length and projected velocity

In [7]:
for i,row in xsections.iterrows():

    vel_dir = f'spot/landslide_{row["id"]}'
    
    if row['id'] == 1:
        vel_dir = vel_dir + '_bird'
    elif row['id'] == 2:
        vel_dir = vel_dir + '_rangitikei'
        
    vel_dir = Path(vel_dir)
    
    _, _, ew_crs = read_raster(vel_dir / 'velocity_ew.tif',crs=True)
    _, _, sn_crs = read_raster(vel_dir / 'velocity_ns.tif',crs=True)
    
    if not ls.crs.to_epsg() == xsections.crs.to_epsg() == ew_crs.to_epsg() == sn_crs.to_epsg():
        raise Exception('EPSGs DO NOT MATCH!!!')
    
    line = row['geometry'].buffer(0.1)
    ew_z = rasterstats.zonal_stats(line,vel_dir / 'velocity_ew.tif', nodata=0.0, stats='mean', all_touched=True)
    sn_z = rasterstats.zonal_stats(line,vel_dir / 'velocity_ns.tif', nodata=0.0, stats='mean', all_touched=True)
    
    ((x1,y1),(x2,y2)) = list(row['geometry'].coords)
    
    angle = np.arctan2(y2 - y1,x2 - x1)
    rotated = angle - (np.pi/2)
    unit_vec = np.array((np.cos(rotated), np.sin(rotated)))

    vel_vec = (ew_z[0]['mean'],sn_z[0]['mean'])
    
    proj_vec = (np.dot(vel_vec, unit_vec)/np.dot(unit_vec, unit_vec))*unit_vec
    proj_mag = np.sqrt((proj_vec[0]**2 + proj_vec[1]**2))
    
    length = row['geometry'].length
    
    ls.loc[ls['id'] == row['id'], 'velocity'] = proj_mag
    ls.loc[ls['id'] == row['id'], 'length'] = length


In [27]:
ls['depth'] = calculate_depth(ls,np.e**-0.73,1.35)
ls['contribution (m3/yr)'] = (ls['depth'] * ls['velocity'] * ls['length']).round()
ls

,id,name,connected,using,notes,geometry,velocity,length,depth,contribution (m3/yr)
0,1,230,0,1,bird landslide,"POLYGON ((399048.042 5612834.973, 399065.275 5...",2.366259,566.767243,47.164821,63254.0
1,2,155,1,1,rangitikei landslide,"POLYGON ((401779.927 5598647.141, 401735.447 5...",1.597453,646.648170,52.052480,53770.0


## validation

In [58]:
rangitikei_annual_volume_loss = 72_000
rangitikei_annual_volume_loss_range = 51_300
rangitikei_annual_volume_loss

72000

In [62]:
bird_data = pd.read_csv('/home/micromamba/data/DoD/DoDV2/InterCompare_t80.csv')
bird_data = bird_data.loc[bird_data['unit'].isin(['head','toe'])].copy()

n_yrs = (datetime(2022,9,1) - datetime(2016,2,1)).days /365.25
total_volume_loss = bird_data.loc[bird_data['unit'] == 'head','volume_thresholded'].mean() * -1
bird_annual_volume_loss = (total_volume_loss / n_yrs).astype(int)
bird_annual_volume_loss

54590

## scratch

In [9]:
p1 = (0,0)
p2 = (0,1)
angle = np.arctan2(p2[1] - p1[1],p2[0] - p1[0])
rotated = angle - (np.pi/2)
unit_vec = np.array((np.cos(rotated), np.sin(rotated)))
unit_vec

array([1., 0.])

In [10]:
vel_vec = np.array((5,5))
vel_vec

array([5, 5])

In [11]:
proj_vec = (np.dot(vel_vec, unit_vec)/np.dot(unit_vec, unit_vec))*unit_vec
proj_mag = np.sqrt((proj_vec[0]**2 + proj_vec[1]**2))
proj_mag

5.0